In [ ]:
import pandas as pd
import numpy as np
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Folder containing all CSV files
data_folder = "/content/drive/MyDrive/CICIOT"  # Update this with your actual path

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Load and concatenate all CSV files into one DataFrame
dataframes = [pd.read_csv(os.path.join(data_folder, file)) for file in csv_files]
df = pd.concat(dataframes, ignore_index=True)

print(f"Total records after merging: {df.shape[0]}")


Total records after merging: 5436586


In [ ]:
df.fillna(0, inplace=True)


In [ ]:
# Encode categorical labels into numerical format
encoder = LabelEncoder()
df["label"] = encoder.fit_transform(df["label"])  # Convert attack types into numeric labels


In [ ]:
# Normalize feature values
scaler = StandardScaler()
X = scaler.fit_transform(df.drop("label", axis=1))  # Normalize input features
y = df["label"].values


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(len(np.unique(y)), activation="softmax")  # Softmax for multi-class classification
])

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Reshape input to fit LSTM
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=2, batch_size=1000, validation_data=(X_test_reshaped, y_test))


Epoch 1/2
4350/4350 ━━━━━━━━━━━━━━━━━━━━ 5182s 1s/step - accuracy: 0.6491 - loss: 0.9455 - val_accuracy: 0.8342 - val_loss: 0.4086
Epoch 2/2
4350/4350 ━━━━━━━━━━━━━━━━━━━━ 4955s 1s/step - accuracy: 0.8252 - loss: 0.4187 - val_accuracy: 0.8305 - val_loss: 0.3874


In [ ]:
# Reshape input to fit LSTM
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=30, batch_size=1000, validation_data=(X_test_reshaped, y_test))


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.title("Model Accuracy")

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Model Loss")

plt.show()


In [ ]:
# Predictions
y_pred = np.argmax(model.predict(X_test_reshaped), axis=1)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Display classification report
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


In [ ]:
# Save the model
model.save("lstm_iot_model.h5")
print("Model saved successfully!")
